In [1]:
import chess
from copy import copy
import chess.svg
import tqdm
from tqdm import tqdm
import random
from random import choice
import tensorflow as tf
import numpy as np

In [2]:
# keras model approach
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dense, Input
from tensorflow.keras.activations import relu

In [3]:
# ADDING THE CODE SO THAT TENSORFLOW DOES NOT EAT THE WHOLE GPU MEMORY
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
b = chess.Board()

#b.push_san("e4")
print(b)
print(b.fen())
b.turn

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1


True

In [5]:
'''import gym
import gym_chess
import random

env = gym.make('Chess-v0')
print(dir(env))
env.reset()
done = False

count = 0
while not done:
    print(env.render())
    print(env._board.fen())
    random.sample
    action = random.sample(env.legal_moves,1)
    a,b,done,d = env.step(action[0])
    state = copy(env)
    #count += 1
    #if count ==2:
    
env.close()'''

"import gym\nimport gym_chess\nimport random\n\nenv = gym.make('Chess-v0')\nprint(dir(env))\nenv.reset()\ndone = False\n\ncount = 0\nwhile not done:\n    print(env.render())\n    print(env._board.fen())\n    random.sample\n    action = random.sample(env.legal_moves,1)\n    a,b,done,d = env.step(action[0])\n    state = copy(env)\n    #count += 1\n    #if count ==2:\n    \nenv.close()"

In [6]:
def encode(board):
    '''
    This is to encode a board position into the nn input
    we will have 12 8*8 boards
    first 6 will have the black pieces
    next 6 will have the white pieces
    
    todo::::
    
    we can later include some cues for castling and special moves
    
    
    
    The order on both of them will be 
    
    1 - pawn
    2 - rook
    3 - knights
    4 - bishop
    5 - queen
    6 - king
    
    The final encoding will have the board reversed (we will see it through blacks perspective)
    
    '''
    array = np.zeros((12, 8, 8), dtype=np.int) # these are the boards
    
    for square, piece in board.piece_map().items():
        x = int(square/8)
        y = int(square%8)

        # this is the black piece encoding
        
        if piece.symbol() == "p":
            array[0][x][y] = 1
            
        if piece.symbol() == "r":
            array[1][x][y] = 1
            
        if piece.symbol() == "n":
            array[2][x][y] = 1
            
        if piece.symbol() == "b":
            array[3][x][y] = 1
            
        if piece.symbol() == "q":
            array[4][x][y] = 1
        
        if piece.symbol() == "k":
            array[5][x][y] = 1
            
        # This is the white piece encoding
        
        if piece.symbol() == "P":
            array[6][x][y] = 1
            
        if piece.symbol() == "R":
            array[7][x][y] = 1
            
        if piece.symbol() == "N":
            array[8][x][y] = 1
            
        if piece.symbol() == "B":
            array[9][x][y] = 1
            
        if piece.symbol() == "Q":
            array[10][x][y] = 1
            
        if piece.symbol() == "K":
            array[11][x][y] = 1
    array.resize((8,8,12))
    array = tf.expand_dims(array,0)
    array = array.numpy()
    array = np.array(array,dtype = "float32")
    return array

temp_encode = encode(b)


In [7]:
def model_keras():
    
    inputs = Input(shape=(8,8,12))
    x = Conv2D(32,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(inputs)
    x = Conv2D(32,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    
    x = Conv2D(64,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(64,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    
    x = Conv2D(128,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(128,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    
    x = Conv2D(256,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(256,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    
    x = Flatten()(x)
    
    x = Dense(1000,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(1000,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(1000,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    output = Dense(1,activation='sigmoid',kernel_initializer="glorot_uniform")(x)
    model = Model(inputs=inputs, outputs=output, name="RL_Value_Function")
    
    print(model.summary())
    
    #model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['binary_crossentropy'])
    
    return model
value_function = model_keras()

Model: "RL_Value_Function"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 8, 12)]        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 32)          3488      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 32)          128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)    

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4)

In [9]:
def create_y_true(current_q,current_r,lamb):
    return tf.math.multiply(current_q,lamb) + current_r
def custom_loss(y_true,y_pred):
    return tf.keras.losses.MSE(y_true,y_pred)

In [10]:
@tf.function
def train_step_keras(model,new_state,new_rew,old_q,lamb):
    #print("input shape is ",inputs.shape)
    
    #outputs = np.array([outputs])
    
    #model.fit(x = inputs,y = outputs,batch_size = 1,epochs = 1)
    
    with tf.GradientTape() as tape:
        # logits is the forward pass
        logits = model(new_state, training=True)
        
        loss_value = custom_loss(create_y_true(logits,new_rew,lamb),old_q)
    
    #we retrieve the gradients
    grads = tape.gradient(loss_value, model.trainable_weights)
    
    #THIS IS ONE STEP OF GRAD DESCENT (Minimizes the loss)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))


In [11]:
#for i in range(20):
#    train_step_keras(keras_model , temp_encode , 0 )


In [12]:
from abc import ABC, abstractmethod
from collections import defaultdict
import math
from copy import copy

class MCTS:
    "Monte Carlo tree searcher. First rollout the tree then choose a move."

    def __init__(self, exploration_weight=1):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.children = dict()  # children of each node
        self.exploration_weight = exploration_weight

    def choose(self, node):
        "Choose the best successor of node. (Choose a move in the game)"
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")

        if node not in self.children:
            return node.find_random_child()

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score)

    def do_rollout(self, node):
        "Make the tree one layer better. (Train for one iteration.)"

        path = self._select(node)

        leaf = path[-1]
        self._expand(leaf)
        
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

        
    def _select(self, node):
        "Find an unexplored descendent of `node`"
        path = []

        while True:
            path.append(node)
            if node not in self.children:
                # node is either unexplored or terminal
                return path
            unexplored = self.children[node] - self.children.keys()
            

            
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        "Update the `children` dict with the children of `node`"
        if node in self.children:
            return  # already expanded
        self.children[node] = node.find_children()

    def _simulate(self, node):

        ''' This simulates the current position for upto n moves and then uses 
            the value function to see what the expected reward might be and then
            returns the reward
            
            This also updates the nn for each state as we have the perfect chess model
            and know the reward for each state
        
        '''
        reward = None
        
        for i in range(20):  
            if node.is_terminal():
                reward = node.reward()
                return reward
            prev_q = value_function(encode(node.position),training = False)[0][0]
            
            node = node.find_random_child()
            reward = value_function(encode(node.position))[0][0]
            #train_step_keras(value_function,encode(node.position),reward,prev_q,0.95)
            
        return reward
    
    def _backpropagate(self, path, reward):
        "Send the reward back up to the ancestors of the leaf"
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            reward = reward

    def _uct_select(self, node):
        "Select a child of node, balancing exploration & exploitation"

        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])

        log_N_vertex = math.log(self.N[node])

        def uct(n):
            "Upper confidence bound for trees"
            return self.Q[n] / self.N[n] + self.exploration_weight * math.sqrt(
                log_N_vertex / self.N[n]
            )

        return max(self.children[node], key=uct)




In [13]:


class Node(ABC):
    """
    A representation of a single board state.
    MCTS works by constructing a tree of these Nodes.
    Could be e.g. a chess or checkers board state.
    """
    
    @abstractmethod
    def find_children(self):
        "All possible successors of this board state"
        
        '''basically if there a move and the game is not over then return
        all possible board positions and not just all possible moves.
        '''

    @abstractmethod
    def find_random_child(self):
        "Random successor of this board state (for more efficient simulation)"
        return None

    @abstractmethod
    def is_terminal(self):
        "Returns True if the node has no children"
        return True

    @abstractmethod
    def reward(self):
        "Assumes `self` is terminal node. 1=win, 0=loss, .5=tie, etc"
        return 0

    
class ChessBoard(Node):
    move = []
    reward = []
    def __init__(self,board):
        self.position = board
        self.curr_reward = 0
        
    def find_children(board):
        if board.position.result() != "*":  # If the game is finished then no moves can be made

            return []
        
        all_next_states = []
        temp_fen = board.position.fen()
        for i in board.position.legal_moves:
            
            board_copy = copy(board)
            board_copy.position.set_fen(temp_fen)
            
            all_next_states.append(board_copy.make_move(board_copy,i))

        return all_next_states
        
    def make_move(self,board1, move):
        board1.position.push(move)
        return board
    
    def find_random_child(board):
        '''Random successor of this board state (for more efficient simulation)
        
        Need a NN named value_function
        and a function encode which converts the board state into the nn format
        '''
        
        if board.position.result() != "*":
            return None
        else:
            current_next_states = []
            current_board_values = []
            max_val = -1000
            max_index = None
            min_val = 100000
            legal = list(board.position.legal_moves)
            temp_fen = board.position.fen()
            for c,i in enumerate(legal):
            # generate a copy of the current board
                board_copy = copy(board)
                board_copy.position.set_fen(temp_fen)
                
                current_next_states.append(board_copy.make_move(board_copy,i).position)
                temp = value_function(encode(board_copy.position),training = False)[0][0]
                current_board_values.append(temp)
                if temp>max_val:
                    max_val = temp
                    max_index = c
                if temp<min_val:
                    min_index = c
            # 10 % exploration chance
            if random.randint(0,9) == 0:
                board_copy = copy(board)
                board_copy.position.set_fen(temp_fen)
                return board_copy.make_move(board_copy,random.sample(legal,1)[0])
            else:
                # 90 % time we choose the best move in SIMULATIONS (this is the simulation policy wrt the value function)
                board_copy = copy(board)
                board_copy.position.set_fen(temp_fen)
                # return best move for white
                if board_copy.position.turn == True:    
                    return board_copy.make_move(board_copy,legal[max_index])
                else:
                    # return best move for black
                    return board_copy.make_move(board_copy,legal[min_index])
            
    def is_terminal(board):
        if board.position.result() == "*":
            return False
        else:
            return True
    def reward(board):
        if board.position.result() == "*":
            return 0
        elif board.position.result() == "0-1":
            return -1
        elif board.position.result() == "1-0":
            return 1
        elif board.position.result() == "1/2-1/2":
            return 0.5
        else:
            print(board.position.result())
            return board.position.result()

## THIS MCTS IS WRONGG as the implementation of Chessboard node is INCORRECT
## EACH BOARD IN THE CHILDREN LIST SHOULD BE A CHESSBOARD OBJECT AND NOT A BOARD STATE OBJECT NEED TO FIX THIS ASAP

In [14]:
# SELF PLAY
#%%time
#%%svg

def calculate_reward(board):
    if board.position.result() == "*":
        return 0
    elif board.position.result() == "0-1":
        return -1
    elif board.position.result() == "1-0":
        return 1
    elif board.position.result() == "1/2-1/2":
        return 0.5

current_state = []
next_state = []
reward_replay = []
game_no = []
moves = 0
for i in range(1):
    temp_board = chess.Board()
    board = ChessBoard(temp_board)
    done = False
    while not done:
        tree = MCTS()
        temp_fen = board.position.fen()
        current_state.append(temp_fen)
        board_copy = copy(board)
        board_copy.position.set_fen(temp_fen)
        prev_q = value_function(encode(board_copy.position),training = False)[0][0]
        
        for j in tqdm(range(100)):
            board_copy = copy(board)
            board_copy.position.set_fen(temp_fen)
            tree.do_rollout(board_copy)
            
        board_copy = copy(board)
        board_copy.position.set_fen(temp_fen)
        print("THE CHOSEN MOVE")
        current_position = tree.choose(board_copy).position
        next_state.append(current_position.fen())
        print(current_position)
        reward = calculate_reward(board)
        reward_replay.append(reward)
        print(reward)
        game_no.append(i)
        train_step_keras(value_function,encode(current_position),reward,prev_q,0.95)
        
        moves = moves + 1
        
        if board.position.result() != "*":
            print(board.position.result())
            done = True

 41%|████      | 41/100 [05:02<07:22,  7.50s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-d6d27686dc64>", line 35, in <module>
    tree.do_rollout(board_copy)
  File "<ipython-input-12-437ca8f753e1>", line 38, in do_rollout
    reward = self._simulate(leaf)
  File "<ipython-input-12-437ca8f753e1>", line 85, in _simulate
    node = node.find_random_child()
  File "<ipython-input-13-ce6e48395522>", line 82, in find_random_child
    temp = value_function(encode(board_copy.position),training = False)[0][0]
  File "<ipython-input-6-bd1b20fffe0c>", line 72, in encode
    array = tf.expand_dims(array,0)
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow_core\python\util\dispatch.py", line 180, in wrapper
    return target(*args, **kwargs)
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\tensorfl

TypeError: object of type 'NoneType' has no len()

In [ ]:
#TURN BASED MOVES
#%%time
import chess
from random import choice
def calculate_reward(board):
    if board.position.result() == "*":
        return 0
    elif board.position.result() == "0-1":
        return -1
    elif board.position.result() == "1-0":
        return 1
    elif board.position.result() == "1/2-1/2":
        return 0.5
moves = 0
for i in range(1):
    temp_board = chess.Board()
    board = ChessBoard(temp_board)
    done = False
    turn = 0
    while not done:
        if turn == 0:
            tree = MCTS()
            temp_fen = board.position.fen()
            for j in tqdm(range(500)):
                board_copy = copy(board)
                board_copy.position.set_fen(temp_fen)
                tree.do_rollout(board_copy)

            board_copy = copy(board)
            board_copy.position.set_fen(temp_fen)
            print("THE CHOSEN MOVE")
            print(tree.choose(board_copy).position)
            chess.svg.board(board_copy)
            reward = calculate_reward(board)
            print(reward)
            moves = moves + 1
            turn = 1
            if board.position.result() != "*":
                print(board.position.result())
                done = True
        if turn == 1:
            player_move = input("ENTER THE MOVE THAT YOU WOULD LIKE TO PLAY : ")
            player_move = chess.Move.from_uci(player_move)
            
            board.position.push(player_move)
            print("The player moved : ")
            print(board.position)
            turn = 0
            if board.position.result() != "*":
                print(board.position.result())
                done = True